In [4]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score


# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
#!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
#nltk.download('stopwords')

# Definimos algunas stopword que queremos que sean eliminadas
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()


In [36]:

atributos_de_interes = ['numeric']

bow = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2))

bow_transformer = ColumnTransformer(transformers=
        [('bow', bow, 'history_text')])

minmax_transformer = ColumnTransformer(transformers=
        [('scaler', MinMaxScaler(), atributos_de_interes)])

union = FeatureUnion([("bow", bow_transformer), ("minmax", minmax_transformer)])

arr = [['hola javier', 12], 
       ['adios javier', 3],]
df = pd.DataFrame(arr, columns=['history_text', 'numeric'])
print(df)
a = union.fit_transform(df)

   history_text  numeric
0   hola javier       12
1  adios javier        3


In [37]:
pd.DataFrame(a, columns=bow_transformer.get_feature_names() + ['numeric'])

,bow__adio,bow__adio javier,bow__hola,bow__hola javier,bow__javier,numeric
0,0.0,0.0,1.0,1.0,1.0,1.0
1,1.0,1.0,0.0,0.0,1.0,0.0
